<a href="https://colab.research.google.com/github/psygorozco/ML/blob/main/youtube_transcript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
! pip install langchain
! pip install sentence-transformers
! pip install chromadb

In [8]:
from langchain_community.document_loaders import YoutubeLoader

In [9]:
! pip install --upgrade --quiet  youtube-transcript-api

In [13]:
loader = YoutubeLoader.from_youtube_url(
    "https://youtu.be/J1wyK4CcvEE?si=AGE13LXHgxCnMySG", add_video_info=False, language=["es"],
)

In [14]:
loader.load()

[Document(page_content='¿Alguna vez has sido víctima o protagonista\nde una infidelidad? Si es así, sabrás lo doloroso y traumático\nque puede ser este hecho para una relación de pareja. La infidelidad es una de las principales causas\nde ruptura y divorcio en el mundo, y afecta tanto a hombres como a mujeres, independientemente\nde su edad, cultura o nivel socioeconómico. La infidelidad se puede definir como la ruptura\ndel compromiso de exclusividad sexual y/o emocional que existe entre dos personas que\nmantienen una relación de pareja. Existen diferentes tipos de infidelidad, según\nel grado de implicación, la duración, la frecuencia, el motivo y el medio que se utilice\npara cometerla. Asimismo, hay diversos factores que pueden\ninfluir en la aparición de la infidelidad, como la insatisfacción, la rutina, la falta\nde comunicación, la baja autoestima, el estrés, la tentación, el aburrimiento o la venganza. Según un estudio realizado por el Instituto\nFrancés de Opinión Pública (IF

In [15]:
import os
from langchain.document_loaders import YoutubeLoader

def download_youtube_transcript(youtube_url, output_dir=".", output_filename=None):
    """
    Descarga la transcripción de un video de YouTube y la guarda en formato de texto (.txt) en un directorio local.

    Args:
        youtube_url (str): URL del video de YouTube.
        output_dir (str, optional): Directorio donde se guardará el archivo de texto. Por defecto, se usa el directorio actual.
        output_filename (str, optional): Nombre del archivo de texto. Si no se proporciona, se usará el ID del video de YouTube.
    """
    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Cargar la transcripción del video de YouTube
    loader = YoutubeLoader.from_youtube_url(youtube_url, add_video_info=False, language=["es"])
    docs = loader.load()

    # Obtener el ID del video de YouTube
    video_id = loader.video_id

    # Establecer el nombre del archivo de salida
    if output_filename is None:
        output_filename = f"{video_id}.txt"
    output_path = os.path.join(output_dir, output_filename)

    # Guardar la transcripción en formato de texto
    with open(output_path, "w", encoding="utf-8") as f:
        for doc in docs:
            f.write(doc.page_content)

    print(f"Transcripción guardada en: {output_path}")

In [16]:
youtube_url = "https://youtu.be/J1wyK4CcvEE?si=AGE13LXHgxCnMySG"
download_youtube_transcript(youtube_url, output_dir="transcripciones")

Transcripción guardada en: transcripciones/J1wyK4CcvEE.txt


In [17]:
import os
from langchain.document_loaders import YoutubeLoader

def download_youtube_transcripts(youtube_urls, output_dir=".", output_filenames=None):
    """
    Descarga las transcripciones de múltiples videos de YouTube y las guarda en formato de texto (.txt) en un directorio local.

    Args:
        youtube_urls (list): Lista de URLs de los videos de YouTube.
        output_dir (str, optional): Directorio donde se guardarán los archivos de texto. Por defecto, se usa el directorio actual.
        output_filenames (list, optional): Lista de nombres de los archivos de texto. Si no se proporciona, se usarán los IDs de los videos de YouTube.
    """
    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Procesar cada URL de YouTube
    for i, youtube_url in enumerate(youtube_urls):
        # Cargar la transcripción del video de YouTube
        loader = YoutubeLoader.from_youtube_url(youtube_url, add_video_info=False, language=["es"])
        docs = loader.load()

        # Obtener el ID del video de YouTube
        video_id = loader.video_id

        # Establecer el nombre del archivo de salida
        if output_filenames is None:
            output_filename = f"{video_id}.txt"
        else:
            output_filename = output_filenames[i]
        output_path = os.path.join(output_dir, output_filename)

        # Guardar la transcripción en formato de texto
        with open(output_path, "w", encoding="utf-8") as f:
            for doc in docs:
                f.write(doc.page_content)

        print(f"Transcripción guardada en: {output_path}")

In [18]:
youtube_urls = [
    "https://youtu.be/3Smhpbid2YY?si=aDXDPr2ri5mWlzI3",
    "https://youtu.be/j8vYTDJGneU?si=278eEn6SAvFxze5S",
    "https://youtu.be/Z6gk59AeYkI?si=w_WTAX9Tkn132Rs4"
]
download_youtube_transcripts(youtube_urls, output_dir="transcripciones", output_filenames=["video1.txt", "video2.txt", "video3.txt"])

Transcripción guardada en: transcripciones/video1.txt
Transcripción guardada en: transcripciones/video2.txt
Transcripción guardada en: transcripciones/video3.txt


In [25]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter

# Carga los textos
loader = TextLoader('transcripciones/video1.txt')
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Crea los embeddings con un modelo de Hugging Face
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")



In [52]:
db

In [35]:
import os
from google.colab import userdata

# Set up the Hugging Face Hub API token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [51]:
from langchain_community.llms import HuggingFaceHub
import os

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)